<a href="https://colab.research.google.com/github/rafapecino/6-2-23/blob/main/ExtraccionCVS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install scikit-learn
!pip install PyPDF2
!pip install sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 121.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 105.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninsta

In [13]:
import os
import json
import re
import PyPDF2
from sentence_transformers import SentenceTransformer, util

class CVInfoExtractor:
    def __init__(self):
        self.section_patterns = {
            "datos_personales": [
                r"datos\s+personales", r"contacto", r"perfil\s+personal", r"información\s+personal",
                r"sobre\s+mí", r"fecha\s+de\s+nacimiento", r"dirección", r"teléfono", r"correo", r"email"
            ],
            "educacion": [
                r"educación", r"formación\s+académica", r"datos\s+académicos", r"estudios",
                r"titulación", r"grado", r"ciclo\s+formativo", r"centro\s+de\s+formación",
                r"universidad", r"bachillerato", r"licenciatura", r"master", r"doctorado"
            ],
            "experiencia": [
                r"experiencia\s+laboral", r"experiencia\s+profesional", r"historial\s+laboral",
                r"trayectoria\s+profesional", r"trabajo", r"empleos", r"prácticas", r"formación\s+en\s+centros"
            ],
            "habilidades": [
                r"habilidades", r"competencias", r"conocimientos", r"aptitudes", r"destrezas", r"capacidades",
                r"conocimientos\s+técnicos", r"herramientas", r"programas", r"actitudes"
            ],
            "idiomas": [
                r"idiomas", r"lenguajes", r"nivel\s+de\s+idioma", r"ingles", r"inglés", r"español", r"castellano",
                r"nativo", r"b1", r"b2", r"c1", r"c2", r"francés", r"alemán", r"italiano", r"portugués"
            ],
            "certificaciones": [
                r"certificaciones", r"certificados", r"cursos", r"curso\s+de", r"diplomas", r"acreditaciones",
                r"especialización", r"diploma"
            ],
            "otros": [
                r"hobbies", r"aficiones", r"intereses", r"voluntariado", r"referencias",
                r"disponibilidad", r"carnet", r"permiso\s+de\s+conducir", r"carnet\s+de\s+conducir", r"coche\s+propio"
            ]
        }

    def extract_text_from_pdf(self, pdf_path):
        try:
            with open(pdf_path, 'rb') as file:
                reader = PyPDF2.PdfReader(file)
                return " ".join(page.extract_text() or "" for page in reader.pages) # Corrected indentation and logic
        except Exception as e:
            print(f"Error al procesar el PDF {pdf_path}: {e}")
            return ""

    def preprocess_text(self, text):
        text = re.sub(r'\s+', ' ', text)
        return re.sub(r'[^\w\s\d\-áéíóúÁÉÍÓÚñÑüÜ@.,:;+()/%]', '', text)

    def split_into_sections(self, text):
        section_headers = [
            r'\b(?:DATOS PERSONALES|PERFIL|SOBRE MÍ)\b',
            r'\b(?:EDUCACIÓN|FORMACIÓN|ESTUDIOS|DATOS ACADÉMICOS)\b',
            r'\b(?:EXPERIENCIA|HISTORIA LABORAL|EMPLEOS|PRÁCTICAS)\b',
            r'\b(?:HABILIDADES|COMPETENCIAS|CONOCIMIENTOS|APTITUDES)\b',
            r'\b(?:IDIOMAS|LENGUAJES)\b',
            r'\b(?:CERTIFICACIONES|CURSOS|DIPLOMAS)\b',
            r'\b(?:INFORMACIÓN ADICIONAL|OTROS|AFICIONES|REFERENCIAS)\b'
        ]
        markers = []
        for pattern in section_headers:
            for match in re.finditer(pattern, text, re.IGNORECASE):
                markers.append((match.start(), match.group()))
        markers.sort()
        sections = []
        for i in range(len(markers)):
            start = markers[i][0]
            end = markers[i+1][0] if i < len(markers) - 1 else len(text)
            sections.append((text[start:end].strip(), start, end))
        if not sections:
            sections.append((text, 0, len(text)))
        return sections

    def classify_section(self, section_text):
        scores = {}
        section_lower = section_text.lower()
        for category, patterns in self.section_patterns.items():
            scores[category] = sum(len(re.findall(p, section_lower, re.IGNORECASE)) for p in patterns)
        best_category = max(scores.items(), key=lambda x: x[1])
        if best_category[1] == 0:
            if re.search(r'\d{4}-\d{4}|\d{4} - \d{4}|\d{2}/\d{2}/\d{4}', section_lower):
                if re.search(r'universidad|colegio|escuela ', section_lower):
                    return "educacion", 1
                return "experiencia", 1
            if re.search(r'python|java|c\+\+|html|css|javascript', section_lower):
                return "habilidades", 1
            if re.search(r'inglés|español|francés|alemán', section_lower):
                return "idiomas", 1
            return "otros", 0
        return best_category[0], best_category[1]

    def extract_structured_info(self, text):
        processed_text = self.preprocess_text(text)
        sections = self.split_into_sections(processed_text)
        categorized_sections = {}
        for section_text, _, _ in sections:
            category, _ = self.classify_section(section_text)
            categorized_sections.setdefault(category, []).append(section_text)
        result = {}
        for category in self.section_patterns.keys():
            result[category] = "\n\n".join(categorized_sections.get(category, ["No se encontró información"]))
        return result

    def save_results(self, results, output_path):
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(results, f, ensure_ascii=False, indent=4)
        print(f"Resultados guardados en: {output_path}")

    def process_cv(self, pdf_path):
        text = self.extract_text_from_pdf(pdf_path)
        if not text:
            return {"error": "No se pudo extraer texto del PDF"}
        return self.extract_structured_info(text)

# Inicializar modelo de embeddings semánticos
model = SentenceTransformer('all-mpnet-base-v2')

def calcular_match_semantico(cv_data, requisitos_texto):
    if not requisitos_texto:
        return 0.0
    cv_texto = "\n".join([cv_data.get(k, "") for k in ["educacion", "experiencia", "habilidades", "idiomas"]])
    emb_cv = model.encode(cv_texto, convert_to_tensor=True)
    emb_req = model.encode(requisitos_texto, convert_to_tensor=True)
    similarity = util.cos_sim(emb_req, emb_cv).item()
    return round(similarity * 100, 2)

def definir_puesto_completo():
    print("\nIntroduce la descripción completa del puesto:")
    return input("> ")

def definir_requisitos_estructurados():
    requisitos = {}
    print("\nIntroduce los requisitos por categoría.")
    requisitos["educacion"] = input("Educación requerida: ")
    requisitos["experiencia"] = input("Experiencia requerida: ")
    requisitos["habilidades"] = input("Habilidades requeridas: ")
    requisitos["idiomas"] = input("Idiomas requeridos: ")
    return "\n".join([f"{k}: {v}" for k, v in requisitos.items()])

def menu():
    mejores_candidatos = []
    extractor = CVInfoExtractor()
    output_folder = "resultados/"
    os.makedirs(output_folder, exist_ok=True)

    print("=== Sistema de Matching de Candidatos ===")
    print("1. Definir un puesto con una descripción completa")
    print("2. Definir requisitos separados por categoría")
    opcion = input("Selecciona una opción (1/2): ")

    if opcion == "1":
        requisitos_texto = definir_puesto_completo()
    elif opcion == "2":
        requisitos_texto = definir_requisitos_estructurados()
    else:
        print("Opción no válida.")
        return

    archivos_cv = [f for f in os.listdir(output_folder) if f.endswith(".json")]
    if not archivos_cv:
        print("No se encontraron resultados procesados en la carpeta 'resultados/'.")
        return

    print("\nResultados de Matching:\n") # Corrected indentation of the print statement
    ranking = []
    for archivo in archivos_cv:
        with open(os.path.join(output_folder, archivo), "r", encoding="utf-8") as f:
            cv_data = json.load(f)
            porcentaje = calcular_match_semantico(cv_data, requisitos_texto)
            print(f"{archivo}: {porcentaje}% de coincidencia") # Corrected indentation
        ranking.append((archivo, porcentaje))

    ranking.sort(key=lambda x: x[1], reverse=True) # Corrected indentation

    print("\nTop candidatos:") # Corrected indentation
    for i, (nombre, score) in enumerate(ranking[:3], 1): # Corrected indentation
        print(f"{i}. {nombre} - {score}%") # Corrected indentation


def procesar_cvs_en_carpeta():
    pdf_folder = "cvs/"
    output_folder = "resultados/"
    os.makedirs(output_folder, exist_ok=True)
    extractor = CVInfoExtractor()

    for filename in os.listdir(pdf_folder):
        if filename.endswith(".pdf"):
            pdf_path = os.path.join(pdf_folder, filename)
            output_path = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}_analisis.json")
            print(f"Procesando: {filename}")
            results = extractor.process_cv(pdf_path)
            extractor.save_results(results, output_path)

if __name__ == "__main__":
    procesar_cvs_en_carpeta()
    menu()

Procesando: Curriculum Vitae Rafael Pecino.pdf
Resultados guardados en: resultados/Curriculum Vitae Rafael Pecino_analisis.json
Procesando: cv-espanol-confoto.pdf
Resultados guardados en: resultados/cv-espanol-confoto_analisis.json
=== Sistema de Matching de Candidatos ===
1. Definir un puesto con una descripción completa
2. Definir requisitos separados por categoría
Selecciona una opción (1/2): 1

Introduce la descripción completa del puesto:
> Título del puesto: Técnico de Soporte y Desarrollo Junior  Descripción: Buscamos un profesional con formación en Ingeniería Informática o similar, con experiencia en soporte técnico y desarrollo de aplicaciones, para incorporarse a nuestro equipo IT. Su responsabilidad será participar en el mantenimiento de sistemas, resolución de incidencias de usuarios, automatización de procesos y tareas de desarrollo en proyectos internos.  Responsabilidades:  Brindar soporte técnico a usuarios internos (hardware, software, redes).  Automatizar tareas media

In [9]:
import os
import json
import re
import PyPDF2
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

class CVInfoExtractor:
    def __init__(self):
        self.section_patterns = {
            "datos_personales": [
                r"datos\s+personales", r"contacto", r"perfil\s+personal",
                r"información\s+personal", r"sobre\s+mí", r"fecha\s+de\s+nacimiento",
                r"dirección", r"teléfono", r"correo", r"email"
            ],
            "educacion": [
                r"educación", r"formación\s+académica", r"datos\s+académicos",
                r"estudios", r"titulación", r"grado", r"ciclo\s+formativo",
                r"centro\s+de\s+formación", r"universidad", r"bachillerato",
                r"licenciatura", r"master", r"doctorado"
            ],
            "experiencia": [
                r"experiencia\s+laboral", r"experiencia\s+profesional",
                r"historial\s+laboral", r"trayectoria\s+profesional",
                r"trabajo", r"empleos", r"prácticas", r"formación\s+en\s+centros"
            ],
            "habilidades": [
                r"habilidades", r"competencias", r"conocimientos", r"aptitudes",
                r"destrezas", r"capacidades", r"conocimientos\s+técnicos",
                r"herramientas", r"programas", r"actitudes"
            ],
            "idiomas": [
                r"idiomas", r"lenguajes", r"nivel\s+de\s+idioma", r"ingles", r"inglés",
                r"español", r"castellano", r"nativo", r"b1", r"b2", r"c1", r"c2",
                r"francés", r"alemán", r"italiano", r"portugués"
            ],
            "certificaciones": [
                r"certificaciones", r"certificados", r"cursos", r"curso\s+de",
                r"diplomas", r"acreditaciones", r"especialización", r"diploma"
            ],
            "otros": [
                r"hobbies", r"aficiones", r"intereses", r"voluntariado",
                r"referencias", r"disponibilidad", r"carnet", r"permiso\s+de\s+conducir",
                r"carnet\s+de\s+conducir", r"coche+propio"
            ]
        }

    def extract_text_from_pdf(self, pdf_path):
        try:
            with open(pdf_path, 'rb') as file:
                reader = PyPDF2.PdfReader(file)
                return " ".join([page.extract_text() or "" for page in reader.pages])
        except Exception as e:
            print(f"Error al procesar el PDF {pdf_path}: {e}")
            return ""

    def preprocess_text(self, text):
        text = re.sub(r'\s+', ' ', text)
        return re.sub(r'[^\w\s\d\-\u00E0-\u00FC@.,:;+()/%]', '', text)

    def split_into_sections(self, text):
        section_headers = [
            r'\b(?:DATOS PERSONALES|PERFIL|SOBRE MÍ)\b',
            r'\b(?:EDUCACIÓN|FORMACIÓN|ESTUDIOS|DATOS ACADÉMICOS)\b',
            r'\b(?:EXPERIENCIA|HISTORIA LABORAL|EMPLEOS|PRÁCTICAS)\b',
            r'\b(?:HABILIDADES|COMPETENCIAS|CONOCIMIENTOS|APTITUDES)\b',
            r'\b(?:IDIOMAS|LENGUAJES)\b',
            r'\b(?:CERTIFICACIONES|CURSOS|DIPLOMAS)\b',
            r'\b(?:INFORMACIÓN ADICIONAL|OTROS|AFICIONES|REFERENCIAS)\b'
        ]

        section_markers = [(m.start(), m.group()) for p in section_headers for m in re.finditer(p, text, re.IGNORECASE)]
        section_markers.sort()

        if not section_markers:
            return [(text, 0, len(text))]

        return [(text[section_markers[i][0]:section_markers[i+1][0] if i+1 < len(section_markers) else len(text)], section_markers[i][0], section_markers[i+1][0] if i+1 < len(section_markers) else len(text)) for i in range(len(section_markers))]

    def classify_section(self, section_text):
        scores = {}
        section_lower = section_text.lower()

        for category, patterns in self.section_patterns.items():
            scores[category] = sum(len(re.findall(p, section_lower, re.IGNORECASE)) for p in patterns)

        best_category = max(scores.items(), key=lambda x: x[1])

        if best_category[1] == 0:
            if re.search(r'\d{4}-\d{4}|\d{4} - \d{4}|\d{2}/\d{2}/\d{4}', section_lower):
                if re.search(r'universidad|colegio|escuela ', section_lower):
                    return "educacion", 1
                return "experiencia", 1
            if re.search(r'python|java|c\+\+|html|css|javascript', section_lower):
                return "habilidades", 1
            if re.search(r'inglés|español|francés|alemán', section_lower):
                return "idiomas", 1
            return "otros", 0

        return best_category[0], best_category[1]

    def extract_structured_info(self, text):
        processed_text = self.preprocess_text(text)
        sections = self.split_into_sections(processed_text)

        categorized_sections = {}
        for section_text, _, _ in sections:
            category, _ = self.classify_section(section_text)
            categorized_sections.setdefault(category, []).append(section_text)

        return {k: "\n\n".join(v) for k, v in categorized_sections.items()}

    def process_cv(self, pdf_path):
        text = self.extract_text_from_pdf(pdf_path)
        if not text:
            return {"error": "No se pudo extraer texto del PDF"}
        return self.extract_structured_info(text)

def calcular_match(cv_data, requisitos_texto):
    if not requisitos_texto:
        return 0.0
    cv_texto = "\n".join([v for k, v in cv_data.items() if k in ["educacion", "experiencia", "habilidades", "idiomas"]])
    corpus = [requisitos_texto, cv_texto]
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(corpus)
    similarity = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])[0][0]
    return round(similarity * 100, 2)

def definir_puesto_completo():
    print("\nIntroduce la descripción completa del puesto (puedes pegar desde un anuncio):")
    return input("Descripción del puesto:\n> ")

def definir_requisitos_estructurados():
    requisitos = {}
    print("\nIntroduce los requisitos por categoría.")
    requisitos["educacion"] = input("Educación requerida: ")
    requisitos["experiencia"] = input("Experiencia requerida: ")
    requisitos["habilidades"] = input("Habilidades requeridas: ")
    requisitos["idiomas"] = input("Idiomas requeridos: ")
    return "\n".join([f"{k}: {v}" for k, v in requisitos.items()])

def menu():
    extractor = CVInfoExtractor()
    pdf_folder = "cvs/"
    output_folder = "resultados/"
    os.makedirs(output_folder, exist_ok=True)

    print("=== Sistema de Matching de Candidatos ===")
    print("1. Definir un puesto con una descripción completa")
    print("2. Definir requisitos separados por categoría")
    opcion = input("Selecciona una opción (1/2): ")

    if opcion == "1":
        requisitos_texto = definir_puesto_completo()
    elif opcion == "2":
        requisitos_texto = definir_requisitos_estructurados()
    else:
        print("Opción no válida.")
        return

    for filename in os.listdir(pdf_folder):
        if filename.endswith(".pdf"):
            pdf_path = os.path.join(pdf_folder, filename)
            output_path = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}_analisis.json")
            print(f"Procesando: {filename}")
            results = extractor.process_cv(pdf_path)
            with open(output_path, 'w', encoding='utf-8') as f:
                json.dump(results, f, ensure_ascii=False, indent=4)

    archivos_cv = [f for f in os.listdir(output_folder) if f.endswith(".json")]
    print("\nResultados de Matching:\n")
    for archivo in archivos_cv:
        with open(os.path.join(output_folder, archivo), "r", encoding="utf-8") as f:
            cv_data = json.load(f)
            porcentaje = calcular_match(cv_data, requisitos_texto)
            print(f"{archivo}: {porcentaje}% de coincidencia")

if __name__ == "__main__":
    menu()

=== Sistema de Matching de Candidatos ===
1. Definir un puesto con una descripción completa
2. Definir requisitos separados por categoría
Selecciona una opción (1/2): 2

Introduce los requisitos por categoría.


KeyboardInterrupt: Interrupted by user

In [10]:
import os
import json
import PyPDF2
import re

class CVInfoExtractor:
    """
    Extractor de información específica de CVs que separa con precisión
    las diferentes secciones usando patrones comunes y contexto
    """

    def __init__(self):
        """Inicializa el extractor con patrones para reconocer secciones"""

        # Patrones para identificar secciones
        self.section_patterns = {
            "datos_personales": [
                r"datos\s+personales", r"contacto", r"perfil\s+personal",
                r"información\s+personal", r"sobre\s+mí", r"fecha\s+de\s+nacimiento",
                r"dirección", r"teléfono", r"correo", r"email"
            ],
            "educacion": [
                r"educación", r"formación\s+académica", r"datos\s+académicos",
                r"estudios", r"titulación", r"grado", r"ciclo\s+formativo",
                r"centro\s+de\s+formación", r"universidad", r"bachillerato",
                r"licenciatura", r"master", r"doctorado"
            ],
            "experiencia": [
                r"experiencia\s+laboral", r"experiencia\s+profesional",
                r"historial\s+laboral", r"trayectoria\s+profesional",
                r"trabajo", r"empleos", r"prácticas", r"formación\s+en\s+centros"
                r"experiencia\s+laboral", r"experiencia\s+profesional",
                r"historial\s+laboral", r"trayectoria\s+profesional",
                r"trabajo", r"empleos", r"prácticas", r"formación\s+en\s+centros"
            ],
            "habilidades": [
                r"habilidades", r"competencias", r"conocimientos", r"aptitudes",
                r"destrezas", r"capacidades", r"conocimientos\s+técnicos",
                r"herramientas", r"programas", r"actitudes"
            ],
            "idiomas": [
                r"idiomas", r"lenguajes", r"nivel\s+de\s+idioma", r"ingles", r"inglés",
                r"español", r"castellano", r"nativo", r"b1", r"b2", r"c1", r"c2"
                r"francés", r"alemán", r"italiano", r"portugués"
            ],
            "certificaciones": [
                r"certificaciones", r"certificados", r"cursos", r"curso\s+de",
                r"diplomas", r"acreditaciones", r"especialización", r"diploma"

            ],
            "otros": [
                r"hobbies", r"aficiones", r"intereses", r"voluntariado",
                r"referencias", r"disponibilidad", r"carnet", r"permiso\s+de\s+conducir", r"carnet\s+de\s+conducir",
                r"coche+propio"
            ]
        }

    def extract_text_from_pdf(self, pdf_path):
        """
        Extrae texto de un archivo PDF

        Args:
            pdf_path: Ruta al archivo PDF

        Returns:
            str: Texto extraído del PDF
        """
        try:
            with open(pdf_path, 'rb') as file:
                reader = PyPDF2.PdfReader(file)
                text = ""
                for page in reader.pages:
                    text += page.extract_text()
            return text
        except Exception as e:
            print(f"Error al procesar el PDF {pdf_path}: {e}")
            return ""

    def preprocess_text(self, text):
        """
        Preprocesa el texto para una mejor extracción

        Args:
            text: Texto del CV

        Returns:
            str: Texto preprocesado
        """
        # Reemplazar múltiples espacios por uno solo
        text = re.sub(r'\s+', ' ', text)

        # Eliminar caracteres especiales pero mantener acentos
        text = re.sub(r'[^\w\s\d\-áéíóúÁÉÍÓÚñÑüÜ@.,:;+()/%]', '', text)

        return text

    def split_into_sections(self, text):
        """
        Divide el texto en secciones basadas en separadores comunes

        Args:
            text: Texto del CV

        Returns:
            list: Lista de secciones [(texto, índice_inicio, índice_fin)]
        """
        # Buscar patrones que indican el inicio de una sección
        section_markers = []

        # Patrones comunes de secciones en CVs
        section_headers = [
            r'\b(?:DATOS PERSONALES|PERFIL|SOBRE MÍ)\b',
            r'\b(?:EDUCACIÓN|FORMACIÓN|ESTUDIOS|DATOS ACADÉMICOS)\b',
            r'\b(?:EXPERIENCIA|HISTORIA LABORAL|EMPLEOS|PRÁCTICAS)\b',
            r'\b(?:HABILIDADES|COMPETENCIAS|CONOCIMIENTOS|APTITUDES)\b',
            r'\b(?:IDIOMAS|LENGUAJES)\b',
            r'\b(?:CERTIFICACIONES|CURSOS|DIPLOMAS)\b',
            r'\b(?:INFORMACIÓN ADICIONAL|OTROS|AFICIONES|REFERENCIAS)\b'
        ]

        # Encontrar todas las posibles secciones
        for pattern in section_headers:
            for match in re.finditer(pattern, text, re.IGNORECASE):
                section_markers.append((match.start(), match.group()))

        # Ordenar por posición
        section_markers.sort()

        # Dividir el texto en secciones
        sections = []
        for i in range(len(section_markers)):
            start = section_markers[i][0]
            end = section_markers[i+1][0] if i < len(section_markers) - 1 else len(text)
            section_text = text[start:end].strip()
            sections.append((section_text, start, end))

        # Si no se encontraron secciones, tratar todo el texto como una sección
        if not sections:
            sections.append((text, 0, len(text)))

        return sections

    def classify_section(self, section_text):
        """
        Clasifica una sección según los patrones de coincidencia

        Args:
            section_text: Texto de la sección

        Returns:
            tuple: (categoría, puntuación)
        """
        scores = {}
        section_lower = section_text.lower()

        for category, patterns in self.section_patterns.items():
            score = 0
            for pattern in patterns:
                matches = re.findall(pattern, section_lower, re.IGNORECASE)
                score += len(matches)
            scores[category] = score

        # Determinar la categoría con mayor puntuación
        best_category = max(scores.items(), key=lambda x: x[1])

        # Si no hay coincidencias claras, analizar el contexto
        if best_category[1] == 0:
            # Buscar pistas contextuales
            if re.search(r'\d{4}-\d{4}|\d{4} - \d{4}|\d{2}/\d{2}/\d{4}', section_lower):
                if re.search(r'universidad|colegio|escuela ', section_lower):
                    return "educacion", 1
                return "experiencia", 1

            if re.search(r'python|java|c\+\+|html|css|javascript', section_lower):
                return "habilidades", 1

            if re.search(r'inglés|español|francés|alemán', section_lower):
                return "idiomas", 1

            # Default para secciones sin coincidencias claras
            return "otros", 0

        return best_category[0], best_category[1]

    def extract_structured_info(self, text):
        """
        Extrae y estructura la información del CV en categorías

        Args:
            text: Texto completo del CV

        Returns:
            dict: Información estructurada por categorías
        """
        # Preprocesar el texto
        processed_text = self.preprocess_text(text)

        # Dividir en secciones
        sections = self.split_into_sections(processed_text)

        # Clasificar cada sección
        categorized_sections = {}
        for section_text, _, _ in sections:
            category, score = self.classify_section(section_text)

            if category not in categorized_sections:
                categorized_sections[category] = []

            categorized_sections[category].append(section_text)

        # Si alguna categoría principal está vacía, intentar extraerla del contexto general
        main_categories = ["datos_personales", "educacion", "experiencia", "habilidades", "idiomas"]
        text_lower = text.lower()

        for category in main_categories:
            if category not in categorized_sections:
                extracted = self.extract_specific_info(text_lower, category)
                if extracted:
                    categorized_sections[category] = [extracted]

        # Unificar el texto por categoría
        result = {}
        for category, sections in categorized_sections.items():
            result[category] = "\n\n".join(sections)

        # Asegurar que todas las categorías estén presentes
        for category in self.section_patterns.keys():
            if category not in result:
                result[category] = "No se encontró información"

        return result

    def extract_specific_info(self, text, category):
        """
        Extrae información específica basada en la categoría

        Args:
            text: Texto completo del CV
            category: Categoría a extraer

        Returns:
            str: Información extraída
        """
        result = []

        if category == "datos_personales":
            # Extraer correo electrónico
            email = re.search(r'[\w.+-]+@[\w-]+\.[\w.-]+', text)
            if email:
                result.append(f"Email: {email.group()}")

            # Extraer teléfono
            phone = re.search(r'(?:\+\d{1,3}[-\s]?)?\(?(?:\d{3})?\)?[-\s]?\d{3}[-\s]?\d{2}[-\s]?\d{2}', text)
            if phone:
                result.append(f"Teléfono: {phone.group()}")

            # Extraer fecha de nacimiento
            birth = re.search(r'\d{1,2}[/.-]\d{1,2}[/.-]\d{2,4}', text)
            if birth:
                result.append(f"Fecha de nacimiento: {birth.group()}")

        elif category == "educacion":
            # Buscar patrones de educación
            education = re.findall(r'(?:grado|ciclo|formación|curso|universidad|superior).*?(?:\d{4}|\d{2}-\d{2})', text, re.IGNORECASE)
            result.extend(education)

        elif category == "experiencia":
            # Buscar patrones de experiencia laboral
            experience = re.findall(r'(?:experiencia|trabajo|empresa|laboral).*?(?:\d{4}|\d{2}-\d{2})', text, re.IGNORECASE)
            result.extend(experience)

        elif category == "habilidades":
            # Extraer habilidades (separadas por comas o guiones)
            skills = re.findall(r'(?:conocimientos|habilidades|competencias):?\s+(.*?)(?:\.|$)', text, re.IGNORECASE)
            result.extend(skills)

            # Buscar lenguajes de programación comunes
            prog_langs = re.findall(r'(?:python|java|javascript|html|css|c\+\+|c#|ruby|php|sql|react|angular)[a-z]*', text, re.IGNORECASE)
            if prog_langs:
                result.append("Lenguajes de programación: " + ", ".join(set(prog_langs)))

        elif category == "idiomas":
            # Extraer menciones a idiomas
            languages = re.findall(r'(?:inglés|español|francés|alemán|italiano|portugués)(?::\s*|\s+)(nativo|[abc][1-2]|básico|intermedio|avanzado|fluido)?', text, re.IGNORECASE)
            for lang in languages:
                if isinstance(lang, tuple):
                    result.append(f"{lang[0]}: {lang[1] if lang[1] else 'mencionado'}")
                else:
                    result.append(lang)

        return "\n".join(result) if result else ""

    def process_cv(self, pdf_path):
        """
        Procesa un CV completo

        Args:
            pdf_path: Ruta al archivo PDF

        Returns:
            dict: Información estructurada
        """
        # Extraer texto del PDF
        text = self.extract_text_from_pdf(pdf_path)
        if not text:
            return {"error": "No se pudo extraer texto del PDF"}

        # Extraer información estructurada
        return self.extract_structured_info(text)

    def process_text(self, text):
        """
        Procesa un texto de CV directamente

        Args:
            text: Texto del CV

        Returns:
            dict: Información estructurada
        """
        return self.extract_structured_info(text)

    def save_results(self, results, output_path):
        """
        Guarda los resultados en formato JSON

        Args:
            results: Resultados de la extracción
            output_path: Ruta para guardar el archivo
        """
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(results, f, ensure_ascii=False, indent=4)
        print(f"Resultados guardados en: {output_path}")

def main():
    """Función principal"""
    pdf_folder = "cvs/"
    output_folder = "resultados/"

    # Crear carpeta de salida si no existe
    os.makedirs(output_folder, exist_ok=True)

    # Inicializar extractor
    extractor = CVInfoExtractor()

    # Procesar todos los PDFs en la carpeta
    for filename in os.listdir(pdf_folder):
        if filename.endswith(".pdf"):
            pdf_path = os.path.join(pdf_folder, filename)
            output_path = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}_analisis.json")

            print(f"Procesando: {filename}")
            results = extractor.process_cv(pdf_path)
            extractor.save_results(results, output_path)

if __name__ == "__main__":
    main()



Procesando: Curriculum Vitae Rafael Pecino.pdf
Resultados guardados en: resultados/Curriculum Vitae Rafael Pecino_analisis.json
Procesando: cv-espanol-confoto.pdf
Resultados guardados en: resultados/cv-espanol-confoto_analisis.json


CLASIFICACION DE CVS-RESULTADOS EN JSON


In [ ]:
import os
import json
import re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

class JobMatcher:
    """
    Sistema para encontrar coincidencias entre CVs y descripciones de puesto de trabajo,
    calculando un porcentaje de match basado en las habilidades requeridas, experiencia
    y otros factores relevantes.
    """

    def __init__(self):
        """Inicializa el sistema de matching"""
        # Inicializar el extractor de CVs
        self.cv_extractor = CVInfoExtractor()

        # Descargar recursos de NLTK si es necesario (solo la primera vez)
        try:
            nltk.data.find('tokenizers/punkt')
            nltk.data.find('corpora/stopwords')
        except LookupError:
            nltk.download('punkt')
            nltk.download('stopwords')

        # Definir stop words en español
        self.stopwords = set(stopwords.words('spanish'))

        # Definir pesos para cada categoría en el proceso de matching
        self.category_weights = {
            "experiencia": 0.35,
            "habilidades": 0.30,
            "educacion": 0.15,
            "idiomas": 0.15,
            "certificaciones": 0.05
        }

        # Definir vectorizador TF-IDF
        self.vectorizer = TfidfVectorizer(
            min_df=1,
            stop_words=self.stopwords,
            lowercase=True,
            analyzer='word',
            ngram_range=(1, 2)  # Unigrams y bigrams
        )

    def parse_job_description(self, job_file):
        """
        Parsea un archivo JSON de descripción de trabajo

        Args:
            job_file: Ruta al archivo JSON con la descripción del puesto

        Returns:
            dict: Información estructurada del puesto
        """
        try:
            with open(job_file, 'r', encoding='utf-8') as f:
                job_data = json.load(f)

            # Verificar que tenga la estructura mínima necesaria
            required_fields = ["titulo", "descripcion", "requisitos"]

            for field in required_fields:
                if field not in job_data:
                    raise ValueError(f"El archivo de trabajo debe contener el campo '{field}'")

            # Estructurar descripción del trabajo en las mismas categorías que los CVs
            structured_job = {
                "titulo": job_data["titulo"],
                "experiencia": self._extract_job_experience(job_data),
                "habilidades": self._extract_job_skills(job_data),
                "educacion": self._extract_job_education(job_data),
                "idiomas": self._extract_job_languages(job_data),
                "certificaciones": self._extract_job_certifications(job_data)
            }

            return structured_job

        except Exception as e:
            print(f"Error al procesar la descripción del trabajo: {e}")
            return None

    def _extract_job_experience(self, job_data):
        """Extrae información sobre experiencia requerida"""
        experience_text = ""

        # Buscar en requisitos
        if "requisitos" in job_data:
            for req in job_data["requisitos"]:
                if any(re.search(r'experi[e|é]ncia|años|laborales', req, re.IGNORECASE)):
                    experience_text += req + " "

        # Buscar en descripción general
        if "descripcion" in job_data:
            exp_matches = re.findall(
                r'(?:se requiere|necesitamos|buscamos|experiencia)(?:.{0,30})(?:años|experiencia)(?:.{0,50})',
                job_data["descripcion"],
                re.IGNORECASE
            )
            experience_text += " ".join(exp_matches)

        return experience_text.strip()

    def _extract_job_skills(self, job_data):
        """Extrae habilidades requeridas para el puesto"""
        skills_text = ""

        # Extraer de requisitos técnicos o habilidades específicas
        if "requisitos" in job_data:
            for req in job_data["requisitos"]:
                if not any(re.search(r'experi[e|é]ncia|años|titulación|idiomas|certificado', req, re.IGNORECASE)):
                    skills_text += req + " "

        # Buscar sección específica de skills o competencias
        if "habilidades" in job_data:
            if isinstance(job_data["habilidades"], list):
                skills_text += " ".join(job_data["habilidades"])
            else:
                skills_text += job_data["habilidades"]

        # Buscar habilidades específicas en la descripción
        skill_patterns = [
            r'conocimientos (?:en|de) ([\w\s,]+)',
            r'habilidades (?:en|de) ([\w\s,]+)',
            r'dominio (?:en|de) ([\w\s,]+)',
            r'manejo (?:en|de) ([\w\s,]+)'
        ]

        if "descripcion" in job_data:
            for pattern in skill_patterns:
                matches = re.findall(pattern, job_data["descripcion"], re.IGNORECASE)
                skills_text += " ".join(matches)

        return skills_text.strip()

    def _extract_job_education(self, job_data):
        """Extrae requisitos educativos"""
        education_text = ""

        # Buscar en requisitos
        if "requisitos" in job_data:
            for req in job_data["requisitos"]:
                if any(re.search(r'grado|licenciatura|titulación|formación|estudios|universidad', req, re.IGNORECASE)):
                    education_text += req + " "

        # Buscar en formación académica
        if "formacion" in job_data:
            if isinstance(job_data["formacion"], list):
                education_text += " ".join(job_data["formacion"])
            else:
                education_text += job_data["formacion"]

        return education_text.strip()

    def _extract_job_languages(self, job_data):
        """Extrae requisitos de idiomas"""
        languages_text = ""

        # Buscar en requisitos
        if "requisitos" in job_data:
            for req in job_data["requisitos"]:
                if any(re.search(r'idiomas?|inglés|francés|alemán|italiano|español|nivel', req, re.IGNORECASE)):
                    languages_text += req + " "

        # Buscar en sección específica de idiomas
        if "idiomas" in job_data:
            if isinstance(job_data["idiomas"], list):
                languages_text += " ".join(job_data["idiomas"])
            else:
                languages_text += job_data["idiomas"]

        return languages_text.strip()

    def _extract_job_certifications(self, job_data):
        """Extrae certificaciones requeridas"""
        certifications_text = ""

        # Buscar en requisitos
        if "requisitos" in job_data:
            for req in job_data["requisitos"]:
                if any(re.search(r'certificado|certificación|título|acreditación|carnet', req, re.IGNORECASE)):
                    certifications_text += req + " "

        # Buscar en descripción general
        if "descripcion" in job_data:
            cert_matches = re.findall(
                r'(?:certificado|certificación|acreditación)(?:\s+en|\s+de)?\s+([\w\s]+)',
                job_data["descripcion"],
                re.IGNORECASE
            )
            certifications_text += " ".join(cert_matches)

        return certifications_text.strip()

    def preprocess_text(self, text):
        """
        Preprocesa el texto para el análisis

        Args:
            text: Texto a preprocesar

        Returns:
            str: Texto preprocesado
        """
        # Convertir a minúsculas
        text = text.lower()

        # Tokenizar
        tokens = word_tokenize(text, language='spanish')

        # Eliminar stopwords y palabras muy cortas
        tokens = [t for t in tokens if t not in self.stopwords and len(t) > 2]

        # Volver a unir en texto
        return ' '.join(tokens)

    def calculate_similarity(self, cv_text, job_text):
        """
        Calcula la similitud entre textos usando TF-IDF y similitud del coseno

        Args:
            cv_text: Texto del currículum
            job_text: Texto de la descripción del trabajo

        Returns:
            float: Valor de similitud entre 0 y 1
        """
        if not cv_text or not job_text:
            return 0.0

        # Preprocesar textos
        cv_processed = self.preprocess_text(cv_text)
        job_processed = self.preprocess_text(job_text)

        if not cv_processed or not job_processed:
            return 0.0

        # Vectorizar textos
        try:
            tfidf_matrix = self.vectorizer.fit_transform([cv_processed, job_processed])

            # Calcular similitud del coseno
            similarity = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])[0][0]

            return max(0.0, min(similarity, 1.0))  # Asegurar que está entre 0 y 1
        except Exception as e:
            print(f"Error al calcular similitud: {e}")
            return 0.0

    def calculate_match(self, cv_data, job_data):
        """
        Calcula el porcentaje de match entre un CV y un puesto de trabajo

        Args:
            cv_data: Datos estructurados del CV
            job_data: Datos estructurados del puesto

        Returns:
            dict: Resultados del match con porcentajes
        """
        match_scores = {}
        total_weight_used = 0

        # Calcular match por categoría
        for category, weight in self.category_weights.items():
            if category in cv_data and category in job_data:
                cv_text = cv_data[category]
                job_text = job_data[category]

                # Si ambos textos existen, calcular similitud
                if cv_text and job_text and cv_text != "No se encontró información":
                    similarity = self.calculate_similarity(cv_text, job_text)
                    match_scores[category] = similarity
                    total_weight_used += weight
                else:
                    match_scores[category] = 0.0
            else:
                match_scores[category] = 0.0

        # Calcular puntaje global ponderado
        weighted_score = sum(
            match_scores[cat] * self.category_weights[cat]
            for cat in self.category_weights.keys()
        )

        # Normalizar si no se usaron todas las categorías
        if total_weight_used > 0 and total_weight_used < 1.0:
            weighted_score /= total_weight_used

        # Crear resultado detallado
        result = {
            "match_porcentaje": round(weighted_score * 100, 2),
            "match_por_categoria": {
                cat: round(score * 100, 2)
                for cat, score in match_scores.items()
            }
        }

        return result

    def find_keyword_matches(self, cv_data, job_data):
        """
        Encuentra coincidencias de palabras clave entre el CV y el puesto

        Args:
            cv_data: Datos estructurados del CV
            job_data: Datos estructurados del puesto

        Returns:
            dict: Palabras clave coincidentes
        """
        # Extraer palabras clave de la descripción del trabajo
        job_keywords = {}

        for category in ["habilidades", "experiencia"]:
            if category in job_data and job_data[category]:
                # Extraer posibles keywords
                text = job_data[category].lower()

                # Palabras clave técnicas (ej. lenguajes de programación, herramientas)
                if category == "habilidades":
                    tech_keywords = re.findall(
                        r'\b(?:python|java|javascript|html|css|angular|react|node|sql|aws|azure|docker|kubernetes|excel|powerbi|tableau|sap|jira|scrum|kanban|agile)\b',
                        text,
                        re.IGNORECASE
                    )
                    job_keywords["tecnologias"] = list(set([kw.lower() for kw in tech_keywords]))

                # Experiencia (ej. años, sectores)
                if category == "experiencia":
                    # Años de experiencia
                    years = re.findall(r'(\d+)(?:\+)?\s*(?:años|year)', text)
                    if years:
                        job_keywords["años_experiencia"] = max([int(y) for y in years])

                    # Sectores o áreas
                    sectors = re.findall(
                        r'(?:sector|industria|área)\s+(?:de|en)?\s+([\w\s]+?)(?:\.|\,|;|$)',
                        text
                    )
                    job_keywords["sectores"] = list(set([s.strip().lower() for s in sectors]))

        # Buscar coincidencias en el CV
        matches = {
            "tecnologias": [],
            "años_experiencia": {"requerido": job_keywords.get("años_experiencia", 0), "encontrado": 0},
            "sectores": []
        }

        # Verificar tecnologías
        if "tecnologias" in job_keywords and "habilidades" in cv_data:
            cv_text = cv_data["habilidades"].lower()
            for tech in job_keywords["tecnologias"]:
                if re.search(r'\b' + re.escape(tech) + r'\b', cv_text, re.IGNORECASE):
                    matches["tecnologias"].append(tech)

        # Verificar años de experiencia
        if "años_experiencia" in job_keywords and "experiencia" in cv_data:
            cv_text = cv_data["experiencia"].lower()
            years_found = re.findall(r'(\d+)(?:\+)?\s*(?:años|year)', cv_text)
            if years_found:
                matches["años_experiencia"]["encontrado"] = max([int(y) for y in years_found])

        # Verificar sectores
        if "sectores" in job_keywords and "experiencia" in cv_data:
            cv_text = cv_data["experiencia"].lower()
            for sector in job_keywords.get("sectores", []):
                if re.search(r'\b' + re.escape(sector) + r'\b', cv_text, re.IGNORECASE):
                    matches["sectores"].append(sector)

        return matches

    def generate_improvement_suggestions(self, cv_data, job_data, match_result):
        """
        Genera sugerencias para mejorar el CV basado en el match

        Args:
            cv_data: Datos estructurados del CV
            job_data: Datos estructurados del puesto
            match_result: Resultados del cálculo de match

        Returns:
            list: Sugerencias de mejora
        """
        suggestions = []

        # Categorías con bajo match
        low_match_categories = [
            cat for cat, score in match_result["match_por_categoria"].items()
            if score < 50 and self.category_weights[cat] >= 0.15  # Solo categorías importantes
        ]

        # Analizar categorías con bajo match
        for category in low_match_categories:
            if category == "habilidades":
                # Extraer habilidades mencionadas en la oferta pero no en el CV
                job_skills = set(self._extract_keywords(job_data["habilidades"], n=15))
                cv_skills = set(self._extract_keywords(cv_data["habilidades"], n=15))

                missing_skills = job_skills - cv_skills
                if missing_skills:
                    skills_str = ", ".join(list(missing_skills)[:5])  # Mostrar hasta 5
                    suggestions.append(
                        f"Añade habilidades relevantes como {skills_str} que se mencionan en la oferta."
                    )

            elif category == "experiencia":
                # Verificar años de experiencia
                job_years = re.search(r'(\d+)(?:\+)?\s*(?:años|year)', job_data["experiencia"])
                cv_years = re.search(r'(\d+)(?:\+)?\s*(?:años|year)', cv_data["experiencia"])

                if job_years and not cv_years:
                    suggestions.append(
                        f"Destaca claramente tus años de experiencia, la oferta menciona {job_years.group(0)}."
                    )

                # Buscar sectores o áreas específicas
                job_sectors = self._extract_keywords(job_data["experiencia"], n=5)
                for sector in job_sectors:
                    if sector not in cv_data["experiencia"].lower():
                        suggestions.append(
                            f"Menciona tu experiencia en '{sector}' si la tienes."
                        )

            elif category == "idiomas":
                if job_data["idiomas"] and (
                    not cv_data["idiomas"] or cv_data["idiomas"] == "No se encontró información"
                ):
                    suggestions.append(
                        "Incluye una sección de idiomas en tu CV, ya que se requieren para este puesto."
                    )
                elif "inglés" in job_data["idiomas"].lower() and "inglés" not in cv_data["idiomas"].lower():
                    suggestions.append(
                        "Menciona tu nivel de inglés, ya que se requiere para este puesto."
                    )

            elif category == "educacion":
                # Verificar requisitos de educación
                job_edu_keywords = ["grado", "licenciatura", "máster", "doctorado", "universitario"]
                job_edu_level = None

                for kw in job_edu_keywords:
                    if kw in job_data["educacion"].lower():
                        job_edu_level = kw
                        break

                if job_edu_level and job_edu_level not in cv_data["educacion"].lower():
                    suggestions.append(
                        f"El puesto requiere nivel de {job_edu_level}. Asegúrate de destacar tu formación académica."
                    )

        # Sugerencias generales basadas en el match global
        if match_result["match_porcentaje"] < 40:
            suggestions.append(
                "Tu CV y este puesto tienen pocas coincidencias. Considera adaptar más tu CV a los requisitos específicos."
            )
        elif match_result["match_porcentaje"] < 60:
            suggestions.append(
                "Adapta la terminología de tu CV para que coincida mejor con la usada en la descripción del puesto."
            )

        return suggestions

    def _extract_keywords(self, text, n=10):
        """
        Extrae las palabras clave más relevantes de un texto

        Args:
            text: Texto a analizar
            n: Número de palabras clave a extraer

        Returns:
            list: Lista de palabras clave
        """
        if not text:
            return []

        # Preprocesar texto
        processed = self.preprocess_text(text)

        # Usar vectorizador TF-IDF para extraer términos importantes
        try:
            tfidf = TfidfVectorizer(
                max_features=n,
                stop_words=self.stopwords,
                ngram_range=(1, 2)
            )

            tfidf_matrix = tfidf.fit_transform([processed])
            feature_names = tfidf.get_feature_names_out()

            # Ordenar por importancia (TF-IDF)
            tfidf_scores = zip(feature_names, tfidf_matrix.toarray()[0])
            sorted_keywords = sorted(tfidf_scores, key=lambda x: x[1], reverse=True)

            return [word for word, score in sorted_keywords if score > 0]
        except:
            # Fallback simple: dividir por espacios y tomar las primeras n palabras
            words = processed.split()
            return list(set(words))[:n]

    def match_cv_with_job(self, cv_file, job_file):
        """
        Realiza el matching completo entre un CV y un puesto

        Args:
            cv_file: Ruta al CV en PDF
            job_file: Ruta a la descripción del puesto en JSON

        Returns:
            dict: Resultado completo del matching
        """
        # Extraer info del CV
        cv_data = self.cv_extractor.process_cv(cv_file)
        if "error" in cv_data:
            return {"error": cv_data["error"]}

        # Parsear descripción del trabajo
        job_data = self.parse_job_description(job_file)
        if not job_data:
            return {"error": "No se pudo procesar la descripción del trabajo"}

        # Calcular match
        match_result = self.calculate_match(cv_data, job_data)

        # Encontrar coincidencias de palabras clave
        keyword_matches = self.find_keyword_matches(cv_data, job_data)

        # Generar sugerencias
        suggestions = self.generate_improvement_suggestions(cv_data, job_data, match_result)

        # Crear resultado final
        result = {
            "titulo_puesto": job_data["titulo"],
            "match_global": match_result["match_porcentaje"],
            "match_por_categoria": match_result["match_por_categoria"],
            "coincidencias_clave": keyword_matches,
            "sugerencias_mejora": suggestions
        }

        return result

    def batch_process(self, cv_folder, job_file, output_folder):
        """
        Procesa múltiples CVs para un mismo puesto

        Args:
            cv_folder: Carpeta con CVs en PDF
            job_file: Archivo JSON con descripción del puesto
            output_folder: Carpeta para guardar resultados
        """
        # Crear carpeta de salida si no existe
        os.makedirs(output_folder, exist_ok=True)

        # Parsear descripción del trabajo
        job_data = self.parse_job_description(job_file)
        if not job_data:
            print("Error: No se pudo procesar la descripción del trabajo")
            return

        results = []

        # Procesar todos los CVs
        for filename in os.listdir(cv_folder):
            if filename.endswith(".pdf"):
                cv_path = os.path.join(cv_folder, filename)
                cv_name = os.path.splitext(filename)[0]

                print(f"Evaluando match para: {filename}")

                # Extraer info del CV
                cv_data = self.cv_extractor.process_cv(cv_path)
                if "error" in cv_data:
                    print(f"Error al procesar {filename}: {cv_data['error']}")
                    continue

                # Calcular match
                match_result = self.calculate_match(cv_data, job_data)

                # Encontrar coincidencias de palabras clave
                keyword_matches = self.find_keyword_matches(cv_data, job_data)

                # Generar sugerencias
                suggestions = self.generate_improvement_suggestions(cv_data, job_data, match_result)

                # Agregar resultado individual
                result = {
                    "nombre_cv": cv_name,
                    "titulo_puesto": job_data["titulo"],
                    "match_global": match_result["match_porcentaje"],
                    "match_por_categoria": match_result["match_por_categoria"],
                    "coincidencias_clave": keyword_matches,
                    "sugerencias_mejora": suggestions
                }

                results.append(result)

                # Guardar resultado individual
                output_path = os.path.join(output_folder, f"{cv_name}_match.json")
                with open(output_path, 'w', encoding='utf-8') as f:
                    json.dump(result, f, ensure_ascii=False, indent=4)

        # Ordenar resultados por porcentaje de match (mayor a menor)
        results.sort(key=lambda x: x["match_global"], reverse=True)

        # Guardar ranking completo
        ranking_path = os.path.join(output_folder, "ranking_candidatos.json")
        with open(ranking_path, 'w', encoding='utf-8') as f:
            json.dump({
                "puesto": job_data["titulo"],
                "candidatos": results
            }, f, ensure_ascii=False, indent=4)

        print(f"Proceso de matching completado. Ranking guardado en: {ranking_path}")

def main():
    """Función principal"""
    cv_folder = "cvs/"
    job_file = "job_description_example.json"
    output_folder = "resultados_match/"

    # Inicializar matcher
    matcher = JobMatcher()

    # Procesar todos los CVs contra el puesto especificado
    matcher.batch_process(cv_folder, job_file, output_folder)

if __name__ == "__main__":
    main()

AttributeError: 'JobMatcher' object has no attribute 'batch_process'

In [ ]:
import json

# Cargar archivo
with open("job_description_example.json", "r", encoding="utf-8") as f:
    job_data = json.load(f)

# Verifica si 'descripcion' existe y no es None
descripcion = job_data.get("descripcion")
if descripcion is None:
    raise ValueError("La descripción del trabajo es None o no existe")

# Verifica si 'requisitos' es una lista
requisitos = job_data.get("requisitos")
if not isinstance(requisitos, list):
    raise ValueError("Los requisitos deben ser una lista")

# Ejemplo: procesar los requisitos
for req in requisitos:
    print("Requisito:", req)


Requisito: Licenciatura/Grado en Informática, Ingeniería de Software o similar
Requisito: Mínimo 3 años de experiencia con Python en entornos de producción
Requisito: Experiencia sólida con Django o Flask
Requisito: Conocimientos de bases de datos SQL (PostgreSQL, MySQL) y NoSQL (MongoDB)
Requisito: Familiaridad con Docker y herramientas de CI/CD
Requisito: Experiencia en diseño y consumo de APIs RESTful
Requisito: Conocimientos de Git y metodologías ágiles
